<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
import pandas as pd
import plotly.express as px
import sqlalchemy as sa

In [2]:
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432

In [3]:
connection_string = f'postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DBNAME}'
engine = sa.create_engine(connection_string)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies).

In [4]:
# текст запроса
query_3_1 = f'''select count(v.*)
                from vacancies v
             '''
with engine.begin() as conn:
    df_3_1 = pd.read_sql_query(sa.text(query_3_1), conn)

In [5]:
# результат запроса
print('Количество вакансий в базе:', df_3_1.loc[0]['count'])

Количество вакансий в базе: 49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers).

In [6]:
# текст запроса
query_3_2 = f'''select count(e.*)
                from employers e
             '''
with engine.begin() as conn:
    df_3_2 = pd.read_sql_query(sa.text(query_3_2), conn)

In [7]:
# результат запроса
print('Количество работодателей в базе:', df_3_2.loc[0]['count'])

Количество работодателей в базе: 23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [8]:
# текст запроса
query_3_3 = f'''select count(a.*)
                from areas a
             '''
with engine.begin() as conn:
    df_3_3 = pd.read_sql_query(sa.text(query_3_3), conn)

In [9]:
# результат запроса
print('Количество регионов в базе:', df_3_3.loc[0]['count'])

Количество регионов в базе: 1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [10]:
# текст запроса
query_3_4 = f'''select count(i.*)
                from industries i
             '''
with engine.begin() as conn:
    df_3_4 = pd.read_sql_query(sa.text(query_3_4), conn)

In [11]:
# результат запроса
print('Количество отраслей в базе:', df_3_4.loc[0]['count'])

Количество отраслей в базе: 294


# выводы по предварительному анализу данных
1. Количество вакансий в базе: 49197

2. Количество работодателей в базе: 23501

3. Количество регионов в базе: 1362

4. Количество отраслей в базе: 294


# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [12]:
# текст запроса
query_4_1 = f'''select
                a.name area,
                count(v.id) cnt
                from vacancies v
                join areas a on v.area_id = a.id
                group by a.name
                order by cnt desc
             '''
with engine.begin() as conn:
    df_4_1 = pd.read_sql_query(sa.text(query_4_1), conn)
print('Количество (cnt) вакансий в каждом регионе (area) по убыванию')

df_4_1['perc'] = round(df_4_1['cnt'] / sum(df_4_1['cnt']) * 100)

df_4_1

Количество (cnt) вакансий в каждом регионе (area) по убыванию


,area,cnt,perc
0,Москва,5333,11.0
1,Санкт-Петербург,2851,6.0
2,Минск,2112,4.0
3,Новосибирск,2006,4.0
4,Алматы,1892,4.0
...,...,...,...
764,Тарко-Сале,1,0.0
765,Новоаннинский,1,0.0
766,Бирск,1,0.0
767,Сасово,1,0.0


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [13]:
# текст запроса
query_4_2 = f'''select
                    count(v.id) cnt
                from vacancies v
                where v.salary_from is not null or v.salary_to is not null
             '''
with engine.begin() as conn:
    df_4_2 = pd.read_sql_query(sa.text(query_4_2), conn)

In [14]:
# результат запроса
print('Количество вакансий с указанной зарплатой:', df_4_2.loc[0]['cnt'])

Количество вакансий с указанной зарплатой: 24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [15]:
# текст запроса
query_4_3 = f'''select
                    avg(v.salary_from) l,
                    avg(v.salary_to) h
                from vacancies v
             '''
with engine.begin() as conn:
    df_4_3 = pd.read_sql_query(sa.text(query_4_3), conn)

In [16]:
# результат запроса
print('Средние значения для нижней и верхней границы зарплатной вилки:', round(df_4_3.loc[0]['l']), round(df_4_3.loc[0]['h']))

Средние значения для нижней и верхней границы зарплатной вилки: 71065 110537


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [17]:
# текст запроса
query_4_4 = f'''
                select
                    count(v.id) cnt,
                    v.schedule,
                    v.employment
                from vacancies v
                group by v.schedule, v.employment
                order by cnt desc
             '''
with engine.begin() as conn:
    df_4_4 = pd.read_sql_query(sa.text(query_4_4), conn)

In [18]:
# результат запроса
print('Количество (cnt) вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment) по убыванию')
df_4_4['perc'] = round(df_4_4['cnt'] / sum(df_4_4['cnt']) * 100)
df_4_4

Количество (cnt) вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment) по убыванию


,cnt,schedule,employment,perc
0,35367,Полный день,Полная занятость,72.0
1,7802,Удаленная работа,Полная занятость,16.0
2,1593,Гибкий график,Полная занятость,3.0
3,1312,Удаленная работа,Частичная занятость,3.0
4,940,Сменный график,Полная занятость,2.0
5,569,Полный день,Стажировка,1.0
6,367,Вахтовый метод,Полная занятость,1.0
7,347,Полный день,Частичная занятость,1.0
8,312,Гибкий график,Частичная занятость,1.0
9,141,Полный день,Проектная работа,0.0


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта.

In [19]:
# текст запроса
query_4_5 = f'''
                select
                    count(v.id) cnt,
                    v.experience exp
                from vacancies v
                group by exp
                order by cnt
             '''
with engine.begin() as conn:
    df_4_5 = pd.read_sql_query(sa.text(query_4_5), conn)

In [20]:
# результат запроса
print('Количество (cnt) вакансий для поля Требуемый опыт работы (exp) в порядке возрастания')
df_4_5['perc'] = round(df_4_5['cnt'] / sum(df_4_5['cnt']) * 100)
display(df_4_5)

Количество (cnt) вакансий для поля Требуемый опыт работы (exp) в порядке возрастания


,cnt,exp,perc
0,1337,Более 6 лет,3.0
1,7197,Нет опыта,15.0
2,14511,От 3 до 6 лет,29.0
3,26152,От 1 года до 3 лет,53.0


# выводы по детальному анализу вакансий

1. Количество вакансий с заполненной зарплатой: 24073

2. Средние значения для нижней и верхней границы зарплатной вилки: 71065 110537

3.	В каждой второй вакансии не заполнено ни одно поле с зарплатой, что возможно говорит о высоком проценте малоквалифицированных соискателей не имеющих особых требований к величине дохода.

4.	Для тех соискателей, которые заполнили хотя бы одно поле с зарплатой, средние значения для верхней и нижней границы зарплаты отличаются всего на 41 тыс.руб., такой разброс характерен для зарплат обычных специалистов и руководителей нижнего звена.

5. При определении зависимости числа вакансий от сочетания типа рабочего графика и типа трудоустройства выявлен само собой разумеющийся факт максимального запроса соискателей на полный рабочий день и полную занятость. Далее с более чем четырехкратным отставанием следует удалённая работа и полная занятость.

6. При  определении количества вакансий от требуемого опыта работы выяснилось, что более половины претендентов удовлетворяют не очень жестким рамкам стажа от года до 3 лет, и всего 2,5% попадают под стаж более 6 лет.



# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [21]:
# текст запроса
query_5_1 = f'''(select
                    count(v.id) cnt,
                    e.name
                from vacancies v
                join employers e on v.employer_id = e.id
                group by e.name
                order by cnt desc
                limit 1)
                union
                (select
                    count(v.id) cnt,
                    e.name
                from vacancies v
                join employers e on v.employer_id = e.id
                group by e.name
                order by cnt desc
                offset 4 limit 1)
                order by cnt desc
             '''
with engine.begin() as conn:
    df_5_1 = pd.read_sql_query(sa.text(query_5_1), conn)

In [22]:
# результат запроса
df_5_1
print('На первом месте среди работодателей по количеству вакансий', df_5_1.loc[0]['name'], df_5_1.loc[0]['cnt'])
print('На пятом месте среди работодателей по количеству вакансий', df_5_1.loc[1]['name'], df_5_1.loc[1]['cnt'])


На первом месте среди работодателей по количеству вакансий Яндекс 1933
На пятом месте среди работодателей по количеству вакансий Газпром нефть 331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [23]:
# результат запроса
query_5_2_1 = f'''with x as 
                        (
                        select
                            a.name,
                            a.id,
                            count(e.area) as cnt_e
                        from areas a
                        left join employers e on e.area = a.id
                        group by a.name, a.id
                        )
                select
                    x.name,
                    coalesce(x.cnt_e, 0) as cnt_e,
                    coalesce(count(v.area_id), 0) as cnt_v
                from x
                left join vacancies v on x.id = v.area_id
                group by x.name, x.cnt_e
                having coalesce(count(v.area_id), 0) = 0
                order by x.cnt_e desc
                limit 1
            '''
with engine.begin() as conn:
    df_5_2_1 = pd.read_sql_query(sa.text(query_5_2_1), conn)
print('Регион, в котором нет вакансий и есть наибольшее количество работодателей', df_5_2_1.loc[0]['cnt_e'], 'это -', df_5_2_1.loc[0]['name'])

Регион, в котором нет вакансий и есть наибольшее количество работодателей 410 это - Россия


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [24]:
# текст запроса
query_5_3 = f'''
                select
                    e.name,
                    count(distinct v.area_id) cnt_a
                from employers e
                join vacancies v on e.id = v.employer_id
                --right join employers e on e.area = a.id
                --where v.area_id is null
                group by e.name
                order by cnt_a desc
             '''
with engine.begin() as conn:
    df_5_3 = pd.read_sql_query(sa.text(query_5_3), conn)

In [25]:
# результат запроса
print('Компании и количество регионов, в которых он публикует свои вакансии')
display(df_5_3)


Компании и количество регионов, в которых он публикует свои вакансии


,name,cnt_a
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
14761,UniSol,1
14762,UNISTORY LLC,1
14763,UNIT6,1
14764,United Distribution,1


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности.

In [26]:
# текст запроса
query_5_4 = f'''
                select
                    count(e.id) cnt_e
                from employers e
                full join employers_industries ei on ei.employer_id = e.id
                where ei.employer_id is null
             '''
with engine.begin() as conn:
    df_5_4 = pd.read_sql_query(sa.text(query_5_4), conn)

In [27]:
# результат запроса
print('Количества работодателей, у которых не указана сфера деятельности', df_5_4.loc[0]['cnt_e'])

Количества работодателей, у которых не указана сфера деятельности 8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности.

In [28]:
# текст запроса
query_5_5 = f'''
                select
                    count(e.id) cnt_e,
                    e.name
                from employers e
                right join employers_industries ei on ei.employer_id = e.id
                --where ei.employer_id is not null
                group by e.name
                having count(e.id) = 4 
                order by e.name
                offset 2 limit 1
             '''
with engine.begin() as conn:
    df_5_5 = pd.read_sql_query(sa.text(query_5_5), conn)

In [29]:
# результат запроса
print('Компания, находящейся на третьем месте в алфавитном списке компаний, у которых указано четыре сферы деятельности', df_5_5.loc[0]['name'])

Компания, находящейся на третьем месте в алфавитном списке компаний, у которых указано четыре сферы деятельности 2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [30]:
# текст запроса
query_5_6 = f'''
                with x as
                (select
                    ei.industry_id
                from employers_industries ei
                right join employers e on ei.employer_id = e.id)
                select 
                    count(x.industry_id)
                from  x
                join industries i on x.industry_id = i.id
                where i.name = 'Разработка программного обеспечения'
             '''
with engine.begin() as conn:
    df_5_6 = pd.read_sql_query(sa.text(query_5_6), conn)

In [31]:
# результат запроса
print('Количество работодателей в качестве сферы деятельности указана Разработка программного обеспечения', df_5_6.loc[0]['count'])

Количество работодателей в качестве сферы деятельности указана Разработка программного обеспечения 3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8).

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API.

In [32]:
# код для получения списка городов-милионников
import requests
from bs4 import BeautifulSoup
url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'
contents = requests.get(url).text
soup = BeautifulSoup(contents, 'html.parser')
elem = soup.select('#mw-content-text > div.mw-parser-output > table.standard.sortable > tbody > tr > td > a')
rows = soup.select('#mw-content-text > div.mw-parser-output > table.standard.sortable > tbody')
lst = []
for row in rows:
    columns = row.select('tr')
    for column in columns:
        name = column.select('a')[0].text
        lst.append(name)
city = tuple(lst[1:])

In [33]:
# текст запроса
query_5_7 = f'''
                (with x as
                    (select
                        e.name,
                        v.id,
                        v.area_id
                    from employers e
                    join vacancies v on e.id = v.employer_id
                    where e.name = 'Яндекс')
                select
                    a.name,
                    count(x.*)
                from x
                left join areas a on x.area_id = a.id
                where a.name in {city}
                group by x.name, a.name
                order by count(x.*) desc)
                UNION ALL 
                (with y as
                    (with x as
                        (select
                            e.name,
                            v.id,
                            v.area_id
                        from employers e
                        join vacancies v on e.id = v.employer_id
                        where e.name = 'Яндекс')
                    select
                        a.name,
                        count(x.id)::int cnt
                    from x
                    left join areas a on x.area_id = a.id
                    where a.name in {city}
                    group by x.name, a.name)                
                select
                     'Total', sum(y.cnt)
                from y)
             '''
with engine.begin() as conn:
    df_5_7 = pd.read_sql_query(sa.text(query_5_7), conn)

In [34]:
# результат запроса
print('Список городов-миллионников, с количеством вакансий компании «Яндекс»')
df_5_7['perc'] = round(df_5_7['count'] / df_5_7.loc[16]['count'] * 100)
df_5_7

Список городов-миллионников, с количеством вакансий компании «Яндекс»


,name,count,perc
0,Москва,54,11.0
1,Санкт-Петербург,42,9.0
2,Екатеринбург,39,8.0
3,Нижний Новгород,36,7.0
4,Новосибирск,35,7.0
5,Воронеж,32,7.0
6,Краснодар,30,6.0
7,Самара,26,5.0
8,Уфа,26,5.0
9,Казань,25,5.0


# выводы по анализу работодателей
1. На первом месте среди работодателей по количеству вакансий 1933 - Яндекс, на пятом месте среди работодателей по количеству вакансий 331 - Газпром нефть.

2. Регион, в котором нет вакансий и есть наибольшее количество работодателей 410 это - Россия

3. Первой компанией по количеству регионов, где она публикует свои вакансии также является Яндекс 181 регион. 

4. Среди городов миллионников где Яндекс открывает вакансии на первом месте стоит Москва - 11% от общего числа вакансий, далее идут Санкт-Петербург и Екатеринбург - 9% и 8% соотвественно.

5. Количества работодателей, у которых не указана сфера деятельности 8419

6. Компания, находящейся на третьем месте в алфавитном списке компаний, у которых указано четыре сферы деятельности 2ГИС

7. Количество работодателей в качестве сферы деятельности указана Разработка программного обеспечения 3553

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.*


In [35]:
# текст запроса
query_6_1 = f'''
                select
                    count(v.name)
                from vacancies v
                where  (v.name ilike '%data%') or (v.name ilike '%данн%')
             '''
with engine.begin() as conn:
    df_6_1 = pd.read_sql_query(sa.text(query_6_1), conn)

In [36]:
# результат запроса
print('Количество вакансий, имеющих отношение к данным', df_6_1.loc[0]['count'])

Количество вакансий, имеющих отношение к данным 1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста?
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.


In [37]:
query_6_2 = f'''
                with x as
                (select
                    v.name,
                    v.experience,
                    v.employment
                from vacancies v
                where  (v.name ilike '%data scientist%' or v.name ilike '%data science%' or v.name ilike '%исследователь данных%' or v.name ilike '%machine learning%'
                or (v.name ilike '%ML%' and v.name not ilike '%HTML%') or v.name ilike '%машинн%обучен%')
                )
                select
                    count(x.name)
                from x
                where (x.name ilike '%junior%') or (x.experience = 'Нет опыта') or (x.employment = 'Стажировка')
             '''
with engine.begin() as conn:
    df_6_2 = pd.read_sql_query(sa.text(query_6_2), conn)

In [38]:
# результат запроса
print('Количество подходящих вакансий для начинающего дата-сайентиста', df_6_2.loc[0]['count'])

Количество подходящих вакансий для начинающего дата-сайентиста 51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [39]:
# текст запроса
query_6_3 = f'''
                with x as
                (select
                    v.name,
                    v.experience,
                    v.employment,
                    v.key_skills
                from vacancies v
                where  (v.name ilike '%data scientist%'
                or v.name ilike '%data science%'
                or v.name ilike '%исследователь данных%'
                or v.name ilike '%machine learning%'
                or (v.name like '%ML%' and v.name not ilike '%HTML%')
                or v.name ilike '%машинн%обучен%')
                )
                select
                    count(*)
                from x
                where (x.key_skills ilike '%SQL%' or x.key_skills ilike '%postges%')
             '''
with engine.begin() as conn:
    df_6_3 = pd.read_sql_query(sa.text(query_6_3), conn)

In [40]:
# результат запроса
print('Количество DS-вакансий с указанием ключевого навыка SQL или postgres', df_6_3.loc[0]['count'])

Количество DS-вакансий с указанием ключевого навыка SQL или postgres 201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [41]:
# текст запроса
query_6_4 = f'''
                with x as
                (select
                    v.name,
                    v.experience,
                    v.employment,
                    v.key_skills
                from vacancies v
                where  (v.name ilike '%data scientist%'
                or v.name ilike '%data science%'
                or v.name ilike '%исследователь данных%'
                or v.name ilike '%machine learning%'
                or (v.name like '%ML%' and v.name not ilike '%HTML%')
                or v.name ilike '%машинн%обучен%')
                )
                select
                    count(*)
                from x
                where (x.key_skills ilike '%python%')
             '''
with engine.begin() as conn:
    df_6_4 = pd.read_sql_query(sa.text(query_6_4), conn)

In [42]:
# результат запроса
print('Количество DS-вакансий с указанием ключевого навыка Python', df_6_4.loc[0]['count'])

Количество DS-вакансий с указанием ключевого навыка Python 351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [43]:
# текст запроса
query_6_5 = f'''
                with x as
                (select
                    v.name,
                    v.experience,
                    v.employment,
                    v.key_skills
                from vacancies v
                where  (v.name ilike '%data scientist%'
                or v.name ilike '%data science%'
                or v.name ilike '%исследователь данных%'
                or v.name ilike '%machine learning%'
                or (v.name like '%ML%' and v.name not ilike '%HTML%')
                or v.name ilike '%машинн%обучен%')
                )
                select
                    avg(array_length(string_to_array(x.key_skills, '\t'), 1)) aver
                from x
             '''
with engine.begin() as conn:
    df_6_5 = pd.read_sql_query(sa.text(query_6_5), conn)

In [44]:
# результат запроса

print('Количество ключевых навыков в вакансиях для DS в среднем', round(df_6_5.loc[0]['aver'], 2))

Количество ключевых навыков в вакансиях для DS в среднем 6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*).

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа.

In [45]:
# текст запроса
query_6_6 = f'''with y as
                    (with x as
                        (select
                            v.name,
                            v.experience,
                            v.employment,
                            v.key_skills,
                            v.salary_from,
                            v.salary_to
                        from vacancies v
                        where  (v.name ilike '%data scientist%'
                        or v.name ilike '%data science%'
                        or v.name ilike '%исследователь данных%'
                        or v.name ilike '%machine learning%'
                        or (v.name like '%ML%' and v.name not ilike '%HTML%')
                        or v.name ilike '%машинн%обучен%') and (v.salary_from is not null or v.salary_to is not null)
                        )
                    select
                        x.experience exp,
                        (x.salary_from + x.salary_to) / 2 av
                    from x
                    where x.salary_from is not null and x.salary_to is not null
                    union all
                    select
                        x.experience exp,
                        (coalesce(x.salary_from, 0) + coalesce(x.salary_to, 0)) av
                    from x
                    where (x.salary_from is not null or x.salary_to is not null) and not (x.salary_from is not null and x.salary_to is not null)
                    )
                select
                    y.exp,
                    avg(y.av)::int aver
                from y
                group by y.exp
             '''
with engine.begin() as conn:
    df_6_6 = pd.read_sql_query(sa.text(query_6_6), conn)

In [46]:
# результат запроса
display(df_6_6)
print('Зарплата предлагаемая для дата-сайентист с опытом работы от 3 до 6 лет в среднем составляет', df_6_6.loc[0]['aver'], 'рублей')

,exp,aver
0,От 3 до 6 лет,243115
1,От 1 года до 3 лет,139675
2,Нет опыта,74643


Зарплата предлагаемая для дата-сайентист с опытом работы от 3 до 6 лет в среднем составляет 243115 рублей


# выводы по предметному анализу

1. Количество вакансий, имеющих отношение к данным 1771

2. Количество подходящих вакансий для начинающего дата-сайентиста 51, что составляет только 3% от количества вакансий, имеющих отношение к данным

3. Количество DS-вакансий с указанием ключевого навыка SQL или postgres 201 и составляет только 11% от количества вакансий, имеющих отношение к данным, что говорит о специфичности этих навыков.

4. Количество DS-вакансий с указанием ключевого навыка Python 351 и составляет 20% от количества вакансий, имеющих отношение к данным и больше показателя в п.3, что говорит о большей востребованности навыка Python.

5. Количество ключевых навыков в вакансиях для DS в среднем 6.41.

6. Зарплата предлагаемая для дата-сайентист с опытом работы от 3 до 6 лет в среднем составляет 243115 рублей.

7. Количество работодателей в качестве сферы деятельности указана Разработка программного обеспечения 3553. Количество вакансий, имеющих отношение к данным 1771. Учитывая, что DS-отрасль довольно молодая, она успешно соперничает с более старшим направлением в IT.


# Общий вывод по проекту

1. По количеству работодателей лидирует Москва и она же лидер по количеству вакансий, опережая почти в два раза Санкт-Петербург.

In [48]:
query_5_2 = f'''with x as 
                        (
                        select
                            a.name,
                            a.id,
                            count(e.area) as cnt_e
                        from areas a
                        left join employers e on e.area = a.id
                        group by a.name, a.id
                        )
                select
                    x.name,
                    coalesce(x.cnt_e, 0) as cnt_e,
                    coalesce(count(v.area_id), 0) as cnt_v
                from x
                left join vacancies v on x.id = v.area_id
                group by x.name, x.cnt_e
                --having coalesce(count(v.area_id), 0) <> 0
                order by x.cnt_e desc
                --limit 1
            '''
with engine.begin() as conn:
    df_5_2 = pd.read_sql_query(sa.text(query_5_2), conn)

fig = px.treemap(df_5_2[df_5_2['cnt_v'] != 0].head(10), path=['name', 'cnt_e', 'cnt_v'], values='cnt_v', color='cnt_v', color_continuous_scale='Blues', 
                 title='Первая десятка регионов по количеству работодателей (второе значение) в регионах с указанием количества вакансий (третье значение)')
#fig.update_traces(marker=dict(cornerradius=5))
fig.show()


2.	В первой десятке по количеству предлагаемых вакансий находятся четыре IT-компании, две телекоммуникационных компании, две компании из сферы финансовых услуг, одна нефтегазовая компания. В отрыве от всех стоит Яндекс предлагающая 1933 вакансий.

In [50]:
query_5_1_1 = f'''
                select
                    count(v.id) cnt,
                    e.name
                from vacancies v
                join employers e on v.employer_id = e.id
                group by e.name
                order by cnt desc
             '''
with engine.begin() as conn:
    df_5_1_1 = pd.read_sql_query(sa.text(query_5_1_1), conn)

fig = px.treemap(df_5_1_1[df_5_1_1['cnt'] != 0].head(10), path=['name', 'cnt'], values='cnt', color='cnt', color_continuous_scale='Blues', 
                 title='Первая десятка работодателей по количеству вакансий')
#fig.update_traces(marker=dict(cornerradius=5))
fig.show()

3.	Первой компанией по количеству регионов, где она публикует свои вакансии также является Яндекс. 

In [51]:
# результат запроса

fig = px.treemap(df_5_3.head(10), path=['name', 'cnt_a'], values='cnt_a', color='cnt_a', color_continuous_scale='Blues', 
                 title='Компании и количество регионов, в которых он публикует свои вакансии')

#fig.update_traces(marker=dict(cornerradius=5))
fig.show()

4.	При определении зависимости числа вакансий от сочетания типа рабочего графика и типа трудоустройства выявлен само собой разумеющийся факт максимального запроса соискателей на полный рабочий день и полную занятость. Далее с более чем четырехкратным отставанием следует удалённая работа и полная занятость.

In [103]:
fig = px.treemap(df_4_4, path=['schedule', 'employment', 'cnt'], values='cnt', color='cnt', color_continuous_scale='Blues', 
                 title='Количество вакансий для каждого сочетания типа рабочего графика и типа трудоустройства', height=700)
fig.update_traces(marker=dict(cornerradius=5))
fig.show()

5.	При  определении зависимости количества вакансий от требуемого опыта работы выяснилось, что более половины претендентов удовлетворяют не очень жестким рамкам стажа от года до 3 лет, и всего 2,5% попадают под стаж более 6 лет.

In [104]:
fig = px.treemap(df_4_5, path=['exp', 'cnt'], values='cnt', color='cnt', color_continuous_scale='Blues', 
                 title='Количество (cnt) вакансий для поля Требуемый опыт работы (exp)')
fig.update_traces(marker=dict(cornerradius=5))
fig.show()

6.	В первой десятке по количеству сфер деятельности, заявленных в вакансиях, находятся три компании так или иначе связанных с IT-деятельностью Модуль, Простые решения, Эксперт.

In [105]:
# текст запроса
query_5_5_1 = f'''
                select
                    count(e.id) cnt_e,
                    e.name
                from employers e
                right join employers_industries ei on ei.employer_id = e.id
                group by e.name
                order by cnt_e desc
             '''
with engine.begin() as conn:
    df_5_5_1 = pd.read_sql_query(sa.text(query_5_5_1), conn)
    
fig = px.treemap(df_5_5_1.head(10), path=['name', 'cnt_e'], values='cnt_e', color='cnt_e', color_continuous_scale='Blues', 
                 title='Первая десятка компаний по количеству сфер деятельности заявленных в вакансиях')

fig.update_traces(marker=dict(cornerradius=5))
fig.show()    

7. В городах с более чем 20 работодателей существует явная зависимость: больше работодателей – больше вакансий. В Москве вообще возникает ситуация, когда количество вакансий превышает количество, тех кто их предлагает. Но при количестве работодателей менее 20 закономерность нарушается, уступая место значительному разбросу количества вакансий от региона к региону. Эта специфика характерна для стран или небольших городов и наводит на мысль, что работодателями являются в том числе кадровые агентства.

In [106]:
query_5_2 = f'''with x as 
                        (
                        select
                            a.name,
                            a.id,
                            count(e.area) as cnt_e
                        from areas a
                        left join employers e on e.area = a.id
                        group by a.name, a.id
                        )
                select
                    x.name,
                    coalesce(x.cnt_e, 0) as cnt_e,
                    coalesce(count(v.area_id), 0) as cnt_v
                from x
                left join vacancies v on x.id = v.area_id
                group by x.name, x.cnt_e
                --having coalesce(count(v.area_id), 0) <> 0
                order by x.cnt_e desc
                --limit 1
            '''
with engine.begin() as conn:
    df_5_2 = pd.read_sql_query(sa.text(query_5_2), conn)

fig = px.scatter(df_5_2, x='cnt_e', y='cnt_v', hover_name='name', log_x=True, log_y=True, title='Зависимость количества вакансий от количества работодателей в регионе')
fig.show()